File to manually compute the cosign between the vectors of the mutated and WT simulations at simulation time one. This is to make sure that the angle between these vectors is small; if it is large, there must be some problem with aligning the structures.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import StandardScaler

## Frame One

In [2]:
# Load in the first frames from both types of simulations:
traj_new = []
traj_new.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/mdcrd/VCBC-A3F_GLU_mut_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/VCBC-A3F_GLU_mut_nowat.prmtop", frame=0))

traj_new.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_nowat.prmtop", frame=0))

In [3]:
# Load in the starting structure:
starting_struct = md.load_pdb("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_tleap.pdb")

In [4]:
# the simulations time in picosecond:
print(traj_new[0].time, traj_new[1].time)

[5.] [5.]


In [5]:
#remove the atoms that are not a part of the backbone:
traj_new[0].atom_slice(traj_new[0].topology.select('backbone'), inplace=True)
traj_new[1].atom_slice(traj_new[1].topology.select('backbone'), inplace=True)
starting_struct.atom_slice(traj_new[1].topology.select('backbone'), inplace=True)

<mdtraj.Trajectory with 1 frames, 2968 atoms, 178 residues, without unitcells at 0x1554126f8770>

In [6]:
traj_new[0]

<mdtraj.Trajectory with 1 frames, 2968 atoms, 742 residues, without unitcells at 0x15541392bef0>

In [7]:
traj_new[1]

<mdtraj.Trajectory with 1 frames, 2968 atoms, 742 residues, without unitcells at 0x1554120bc0b0>

In [8]:
# align the two trajectories and the starting structure:
traj_new[1].superpose(reference = traj_new[0])
traj_new[0].superpose(reference = traj_new[0])
starting_struct.superpose(reference = traj_new[0])

<mdtraj.Trajectory with 1 frames, 2968 atoms, 178 residues, without unitcells at 0x1554126f8770>

In [9]:
# extract the x,y,z coordinates from the trajectory objects
coordinatesGLU = traj_new[0].xyz
coordinatesWT = traj_new[1].xyz
Startcoordinates = starting_struct.xyz

print(coordinatesGLU.shape, coordinatesWT.shape, Startcoordinates.shape)

(1, 2968, 3) (1, 2968, 3) (1, 2968, 3)


In [10]:
# Flatten to (n_frames, n_atoms * 3)
coordinatesGLU = coordinatesGLU.reshape(coordinatesGLU.shape[0], -1) 
coordinatesWT = coordinatesWT.reshape(coordinatesWT.shape[0], -1)
Startcoordinates = Startcoordinates.reshape(Startcoordinates.shape[0], -1)
print(coordinatesGLU.shape, coordinatesWT.shape, Startcoordinates.shape)

(1, 8904) (1, 8904) (1, 8904)


In [11]:
# Calculate the cosign similarity between the two vectors
# https://datastax.medium.com/how-to-implement-cosine-similarity-in-python-505e8ec1d823#:~:text=Cosine%20similarity%20formula&text=It's%20calculated%20as%20%7C%7CA,way%20as%20%7C%7CA%7C%7C.

cosine_similarity_result1 = cosine_similarity(coordinatesGLU, coordinatesWT)
print(f"Cosine Similarity between WT and GLU: {cosine_similarity_result1[0][0]}")

cosine_similarity_result2 = cosine_similarity(coordinatesGLU, Startcoordinates)
print(f"Cosine Similarity between GLU and the starting structure: {cosine_similarity_result2[0][0]}")

cosine_similarity_result3 = cosine_similarity(coordinatesWT, Startcoordinates)
print(f"Cosine Similarity between WT and the starting structure: {cosine_similarity_result3[0][0]}")


Cosine Similarity between WT and GLU: 0.9996817708015442
Cosine Similarity between GLU and the starting structure: 0.9633190631866455
Cosine Similarity between WT and the starting structure: 0.9645692706108093


From this, it appears that the GLU and WT cosign similarity is very close to one, meaning they are nearly identical. It is interesting that both of the simulations have vectors that differ more from the starting structure, though this could be because the starting frames are from the simulations after they underwent the equillibration process, so they had some time to move away from the starting structure to a more stable conformation.

## Last Frame

I'm also going to compare the cosign similarities from a later point in the simulations to see if the simulations do, in fact, move away from each other and the cosign similarity gets smaller. 

In [12]:
# Load in the last frame from the first independent simulation from both types of simulations:
traj_late = []
traj_late.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/mdcrd/VCBC-A3F_GLU_mut_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/VCBC-A3F_GLU_mut_nowat.prmtop", frame=49999))

traj_late.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_nowat.prmtop", frame=49999))

In [13]:
# the simulations time in picosecond:
print(traj_late[0].time, traj_late[1].time)

[250000.] [250000.]


In [14]:
#remove the atoms that are not a part of the backbone:
traj_late[0].atom_slice(traj_late[0].topology.select('backbone'), inplace=True)
traj_late[1].atom_slice(traj_late[1].topology.select('backbone'), inplace=True)

<mdtraj.Trajectory with 1 frames, 2968 atoms, 742 residues, without unitcells at 0x155412593fe0>

In [15]:
# align the two trajectories and the starting structure:
traj_late[1].superpose(reference = traj_late[0])
traj_late[0].superpose(reference = traj_late[0])
starting_struct.superpose(reference = traj_late[0])

<mdtraj.Trajectory with 1 frames, 2968 atoms, 178 residues, without unitcells at 0x1554126f8770>

In [16]:
# extract the x,y,z coordinates from the trajectory objects
late_coordinatesGLU = traj_late[0].xyz
late_coordinatesWT = traj_late[1].xyz
Startcoordinates = starting_struct.xyz

print(late_coordinatesGLU.shape, late_coordinatesWT.shape, Startcoordinates.shape)

(1, 2968, 3) (1, 2968, 3) (1, 2968, 3)


In [17]:
# Flatten to (n_frames, n_atoms * 3)
late_coordinatesGLU = late_coordinatesGLU.reshape(late_coordinatesGLU.shape[0], -1) 
late_coordinatesWT = late_coordinatesWT.reshape(late_coordinatesWT.shape[0], -1)
Startcoordinates = Startcoordinates.reshape(Startcoordinates.shape[0], -1)
print(late_coordinatesGLU.shape, late_coordinatesWT.shape, Startcoordinates.shape)

(1, 8904) (1, 8904) (1, 8904)


In [18]:
# Calculate the cosign similarity between the two vectors
# https://datastax.medium.com/how-to-implement-cosine-similarity-in-python-505e8ec1d823#:~:text=Cosine%20similarity%20formula&text=It's%20calculated%20as%20%7C%7CA,way%20as%20%7C%7CA%7C%7C.

late_cosine_similarity_result1 = cosine_similarity(late_coordinatesGLU, late_coordinatesWT)
print(f"Cosine Similarity between WT and GLU: {late_cosine_similarity_result1[0][0]}")

late_cosine_similarity_result2 = cosine_similarity(late_coordinatesGLU, Startcoordinates)
print(f"Cosine Similarity between GLU and the starting structure: {late_cosine_similarity_result2[0][0]}")

late_cosine_similarity_result3 = cosine_similarity(late_coordinatesWT, Startcoordinates)
print(f"Cosine Similarity between WT and the starting structure: {late_cosine_similarity_result3[0][0]}")

Cosine Similarity between WT and GLU: 0.9987751841545105
Cosine Similarity between GLU and the starting structure: 0.9629177451133728
Cosine Similarity between WT and the starting structure: 0.9638996124267578


Even after 250000 picoseconds of simulation time, the cosign similarity between the two simulations is very close to one. This likely means the complexes are not diverging from each other in terms of structure, but the similarity could possibly be because I algined the backbone for just these two frames (as opposed to aligning everything to the first frame of the GLU simulation data).

## Middle Frame

In [19]:
# Load in the middle frame from the first independent simulation from both types of simulations:
traj_mid = []
traj_mid.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/mdcrd/VCBC-A3F_GLU_mut_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/VCBC-A3F_GLU_mut_nowat.prmtop", frame=25000))

traj_mid.append(md.load_netcdf("/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_sims1-8_first300_noBox.mdcrd", 
                          top="/data/nvanhoorn/A3F_newstructure/VCBC_A3F_GLU_mut/redoing_everything/WT_files/VCBC-A3F_WT_nowat.prmtop", frame=25000))

In [20]:
# the simulations time in picosecond:
print(traj_mid[0].time, traj_mid[1].time)

[125005.] [125005.]


In [21]:
#remove the atoms that are not a part of the backbone:
traj_mid[0].atom_slice(traj_mid[0].topology.select('backbone'), inplace=True)
traj_mid[1].atom_slice(traj_mid[1].topology.select('backbone'), inplace=True)

<mdtraj.Trajectory with 1 frames, 2968 atoms, 742 residues, without unitcells at 0x1554120b60f0>

In [22]:
# align the two trajectories and the starting structure:
traj_mid[1].superpose(reference = traj_mid[0])
traj_late[0].superpose(reference = traj_late[0])
starting_struct.superpose(reference = traj_mid[0])

<mdtraj.Trajectory with 1 frames, 2968 atoms, 178 residues, without unitcells at 0x1554126f8770>

In [23]:
# extract the x,y,z coordinates from the trajectory objects
mid_coordinatesGLU = traj_mid[0].xyz
mid_coordinatesWT = traj_mid[1].xyz
Startcoordinates = starting_struct.xyz

print(mid_coordinatesGLU.shape, mid_coordinatesWT.shape, Startcoordinates.shape)

(1, 2968, 3) (1, 2968, 3) (1, 2968, 3)


In [24]:
# Flatten to (n_frames, n_atoms * 3)
mid_coordinatesGLU = mid_coordinatesGLU.reshape(mid_coordinatesGLU.shape[0], -1) 
mid_coordinatesWT = mid_coordinatesWT.reshape(mid_coordinatesWT.shape[0], -1)
Startcoordinates = Startcoordinates.reshape(Startcoordinates.shape[0], -1)
print(mid_coordinatesGLU.shape, mid_coordinatesWT.shape, Startcoordinates.shape)

(1, 8904) (1, 8904) (1, 8904)


In [25]:
# Calculate the cosign similarity between the two vectors
# https://datastax.medium.com/how-to-implement-cosine-similarity-in-python-505e8ec1d823#:~:text=Cosine%20similarity%20formula&text=It's%20calculated%20as%20%7C%7CA,way%20as%20%7C%7CA%7C%7C.

mid_cosine_similarity_result1 = cosine_similarity(mid_coordinatesGLU, mid_coordinatesWT)
print(f"Cosine Similarity between WT and GLU: {mid_cosine_similarity_result1[0][0]}")

mid_cosine_similarity_result2 = cosine_similarity(mid_coordinatesGLU, Startcoordinates)
print(f"Cosine Similarity between GLU and the starting structure: {mid_cosine_similarity_result2[0][0]}")

mid_cosine_similarity_result3 = cosine_similarity(mid_coordinatesWT, Startcoordinates)
print(f"Cosine Similarity between WT and the starting structure: {mid_cosine_similarity_result3[0][0]}")

Cosine Similarity between WT and GLU: 0.9995341300964355
Cosine Similarity between GLU and the starting structure: 0.9593423008918762
Cosine Similarity between WT and the starting structure: 0.960163950920105


In [26]:
print(f"Cosine Difference (last-first frame) between WT and GLU: {late_cosine_similarity_result1[0][0] - cosine_similarity_result1[0][0]}")
print(f"Cosine Difference (last-first frame) between GLU and the starting structure: {late_cosine_similarity_result2[0][0] - cosine_similarity_result2[0][0]}")
print(f"Cosine Difference (last-first frame) between WT and the starting structure: {late_cosine_similarity_result3[0][0] - cosine_similarity_result3[0][0]}")

Cosine Difference (last-first frame) between WT and GLU: -0.0009065866470336914
Cosine Difference (last-first frame) between GLU and the starting structure: -0.0004013180732727051
Cosine Difference (last-first frame) between WT and the starting structure: -0.0006696581840515137


In [27]:
print(f"Cosine Difference (mid-first frame) between WT and GLU: {mid_cosine_similarity_result1[0][0] - cosine_similarity_result1[0][0]}")
print(f"Cosine Difference (mid-first frame) between GLU and the starting structure: {mid_cosine_similarity_result2[0][0] - cosine_similarity_result2[0][0]}")
print(f"Cosine Difference (mid-first frame) between WT and the starting structure: {mid_cosine_similarity_result3[0][0] - cosine_similarity_result3[0][0]}")

Cosine Difference (mid-first frame) between WT and GLU: -0.00014764070510864258
Cosine Difference (mid-first frame) between GLU and the starting structure: -0.003976762294769287
Cosine Difference (mid-first frame) between WT and the starting structure: -0.004405319690704346
